In [51]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [84]:
base_url = "https://en.wikipedia.org/wiki/"

words = [
    "BUFFALO", "COW", "GOAT", "SHEEP", 
    "BEAM", "GLOW", "RADIATE", "SHINE",
    "FLOOR", "HORSE", "RINGS", "VAULT",
    "CUTIE", "ENVY", "EXCEL", "SEEDY"
]

In [85]:
# for word in words:
#     print(f'{api_url}{word.capitalize()}')
#     page = requests.get(f'{api_url}{word.capitalize()}')
#     pprint(response.text)

soups = [BeautifulSoup(requests.get(f'{base_url}{word.capitalize()}').content, 'html.parser') for word in words]

In [112]:
wiki_df = pd.DataFrame({"Word": [], "Definition": []})

for i, soup in enumerate(soups):
    content = soup.find("div", id="mw-content-text")
    
    first_para = content.find("p").text
    
    items = [item.text for item in content.find_all("li")]
    
    if f'may refer to:' in first_para:
        for item in items:
            
            upper_letter = item.upper()
            
            while f'{words[i]}s' in upper_letter:
                item = item[upper_letter.index(f'{words[i]}s') + len(f'{words[i]}s'):]
                upper_letter = upper_letter[upper_letter.index(f'{words[i]}s') + len(f'{words[i]}s'):]
            
            while f'{words[i]}ing' in upper_letter:
                item = item[upper_letter.index(f'{words[i]}ing') + len(f'{words[i]}ing'):]
                upper_letter = upper_letter[upper_letter.index(f'{words[i]}ing') + len(f'{words[i]}ing'):]
            
            while f'{words[i]}es' in upper_letter:
                item = item[upper_letter.index(f'{words[i]}es') + len(f'{words[i]}es'):]
                upper_letter = upper_letter[upper_letter.index(f'{words[i]}es') + len(f'{words[i]}es'):]
                
            while f'{words[i]}ed' in upper_letter:
                item = item[upper_letter.index(f'{words[i]}ed') + len(f'{words[i]}ed'):]
                upper_letter = upper_letter[upper_letter.index(f'{words[i]}ed') + len(f'{words[i]}ed'):]
            
            while f'{words[i]}' in upper_letter:
                item = item[upper_letter.index(f'{words[i]}') + len(f'{words[i]}'):]
                upper_letter = upper_letter[upper_letter.index(f'{words[i]}') + len(f'{words[i]}'):]
                
            if bool(item):
                wiki_df.loc[len(wiki_df)] = {"Word": words[i], "Definition": "".join(item)}
            
wiki_df

,Word,Definition
0,BEAM,of particles moving at approximately equal ve...
1,BEAM,", a penetrating form of high-energy electromag..."
2,BEAM,", streams of electrons observed in discharge t..."
3,BEAM,", a penetrating form of high-energy electromag..."
4,BEAM,of particles moving at approximately equal ve...
...,...,...
398,VAULT,"Comics, an American comic book publisher"
399,VAULT,s
400,VAULT,(disambiguation)
401,VAULT,Treasury


In [113]:
dict_df = pd.read_csv("data/dictionary.csv")

dict_df

,Word,POS,Definition
0,A,NaN,The first letter of the English and of many ot...
1,A,NaN,The name of the sixth tone in the model major ...
2,A,NaN,An adjective commonly called the indefinite ar...
3,A,NaN,"In each; to or for each; as """"""""twenty leagues..."
4,A,prep.,In; on; at; by.
...,...,...,...
175718,Zymotic,a.,Of pertaining to or caused by fermentation.
175719,Zymotic,a.,Designating or pertaining to a certain class o...
175720,Zythem,n.,See Zythum.
175721,Zythepsary,n.,A brewery.


In [114]:
dict_df["Word"] = dict_df["Word"].str.upper()

In [115]:
dict_df = dict_df[dict_df["Word"].isin(words)]

dict_df = dict_df.reset_index()
dict_df

,index,Word,POS,Definition
0,13608,BEAM,n.,Any large piece of timber or iron long in prop...
1,13609,BEAM,n.,One of the principal horizontal timbers of a b...
2,13610,BEAM,n.,The width of a vessel; as one vessel is said t...
3,13611,BEAM,n.,The bar of a balance from the ends of which th...
4,13612,BEAM,n.,The principal stem or horn of a stag or other ...
...,...,...,...,...
111,168075,VAULT,n.,A leap by aid of the hands or of a pole spring...
112,168078,VAULT,v. t.,To form with a vault or to cover with a vault;...
113,168079,VAULT,v. i.,To leap over; esp. to leap over by aid of the ...
114,168080,VAULT,n.,To leap; to bound; to jump; to spring.


In [116]:
combined_words = pd.concat([wiki_df["Word"], dict_df["Word"]])
combined_defs = pd.concat([wiki_df["Definition"], dict_df["Definition"]])

wiki_df = pd.DataFrame({"Word": combined_words, "Definition": combined_defs})

wiki_df

,Word,Definition
0,BEAM,of particles moving at approximately equal ve...
1,BEAM,", a penetrating form of high-energy electromag..."
2,BEAM,", streams of electrons observed in discharge t..."
3,BEAM,", a penetrating form of high-energy electromag..."
4,BEAM,of particles moving at approximately equal ve...
...,...,...
111,VAULT,A leap by aid of the hands or of a pole spring...
112,VAULT,To form with a vault or to cover with a vault;...
113,VAULT,To leap over; esp. to leap over by aid of the ...
114,VAULT,To leap; to bound; to jump; to spring.


In [117]:
wiki_df['word_number'] = wiki_df.groupby('Word').cumcount() + 1

wiki_df['Word'] = wiki_df.apply(lambda row: f"{row['Word']}_{row['word_number']}", axis=1)

wiki_df = wiki_df.dropna()

wiki_df

,Word,Definition,word_number
0,BEAM_1,of particles moving at approximately equal ve...,1
1,BEAM_2,", a penetrating form of high-energy electromag...",2
2,BEAM_3,", streams of electrons observed in discharge t...",3
3,BEAM_4,", a penetrating form of high-energy electromag...",4
4,BEAM_5,of particles moving at approximately equal ve...,5
...,...,...,...
111,VAULT_44,A leap by aid of the hands or of a pole spring...,44
112,VAULT_45,To form with a vault or to cover with a vault;...,45
113,VAULT_46,To leap over; esp. to leap over by aid of the ...,46
114,VAULT_47,To leap; to bound; to jump; to spring.,47


In [161]:
import numpy as np
import pandas as pd
import math
import re
from sentence_transformers import SentenceTransformer
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy.linalg import norm

In [162]:
device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'

retriever = SentenceTransformer(
    "paraphrase-MiniLM-L6-v2",
    device = device
)

In [163]:
wiki_df["Definition"] = wiki_df["Definition"].astype(str)
wiki_df.dtypes

Word           object
Definition     object
word_number     int64
dtype: object

In [164]:
embeddings = [retriever.encode(defi) for defi in wiki_df['Definition']]
embeddings = np.array(embeddings)

embeddings.shape

(519, 384)

In [165]:
matrix = embeddings

In [166]:
def cosine_similarity(a, b):
    return np.dot(a,b)/(norm(a)*norm(b))

In [167]:
matrix.shape

(519, 384)

In [168]:
similarities = []

for i in range(len(matrix)):
    a = matrix[i]
    for j in range(i, len(matrix)):
        b = matrix[j]
        word1 = wiki_df.iloc[i]["Word"]
        word2 = wiki_df.iloc[j]["Word"]
        if word1[0: word1.index("_")] != word2[0: word2.index("_")]:
            sim = cosine_similarity(a, b)/math.dist(a, b)
            if math.isinf(sim):
                sim = 1
            similarities.append([wiki_df.iloc[i]["Word"], wiki_df.iloc[j]["Word"], sim])
            
df = pd.DataFrame(similarities, columns=["word_1", "word_2", "similarity"])

df

/var/folders/y6/0cl3w1zj1j39sz13xbl1z24w0000gn/T/ipykernel_4346/2065625144.py:10: RuntimeWarning: divide by zero encountered in divide
  similarities.append([wiki_df.iloc[i]["Word"], wiki_df.iloc[j]["Word"], cosine_similarity(a, b)/math.dist(a, b)])


,word_1,word_2,similarity
0,BEAM_1,GLOW_1,-0.004880
1,BEAM_1,GLOW_2,-0.007473
2,BEAM_1,GLOW_3,0.050762
3,BEAM_1,GLOW_4,0.037785
4,BEAM_1,GLOW_5,0.016561
...,...,...,...
110627,SHINE_168,VAULT_44,0.007824
110628,SHINE_168,VAULT_45,0.012159
110629,SHINE_168,VAULT_46,0.003385
110630,SHINE_168,VAULT_47,0.010487
